In [2]:
from __future__ import annotations

from pathlib import Path
from typing import List, Dict, Any, Optional, Literal, Union
from datetime import datetime

import json
import io
import contextlib

from pydantic import BaseModel, Field

# langchain / langgraph imports
from langchain_core.tools import tool
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage,
    BaseMessage,
    ToolMessage,
)
from langchain_openai import ChatOpenAI

from langgraph.graph import StateGraph, END, MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver

# ===========================
# Paths
# ===========================

PROJECT_ROOT = Path("/scratch/ziv_baretto/llmserve").resolve()
DATA_DIR = PROJECT_ROOT / "data"
STAGE2_OUT_DIR = PROJECT_ROOT / "stage2_out"
STAGE3_OUT_DIR = PROJECT_ROOT / "stage3_out"
STAGE3_OUT_DIR.mkdir(parents=True, exist_ok=True)

print("PROJECT_ROOT :", PROJECT_ROOT)
print("DATA_DIR     :", DATA_DIR)
print("STAGE2_OUT   :", STAGE2_OUT_DIR)
print("STAGE3_OUT   :", STAGE3_OUT_DIR)

# ===========================
# Stage 2 models
# ===========================

TaskCategory = Literal["predictive", "descriptive", "unsupervised"]


class JoinPlan(BaseModel):
    hypothesized_keys: List[List[str]] = Field(default_factory=list)
    notes: Optional[str] = None


class TargetSpec(BaseModel):
    name: Optional[str] = None
    granularity: Optional[List[str]] = None
    horizon: Optional[str] = None


class FeaturePlan(BaseModel):
    candidates: List[str] = Field(default_factory=list)
    transform_ideas: List[str] = Field(default_factory=list)
    handling_missingness: Optional[str] = None


class TaskProposal(BaseModel):
    id: str
    category: TaskCategory
    title: str
    problem_statement: str
    required_files: List[str] = Field(default_factory=list)
    join_plan: JoinPlan = Field(default_factory=JoinPlan)
    target: Optional[TargetSpec] = None
    feature_plan: FeaturePlan = Field(default_factory=FeaturePlan)
    validation_plan: Optional[str] = None
    quality_checks: List[str] = Field(default_factory=list)
    expected_outputs: List[str] = Field(default_factory=list)


class Stage2Output(BaseModel):
    proposals: List[TaskProposal]


# ===========================
# Stage 3 Plan Models
# ===========================


class ArtifactSpec(BaseModel):
    intermediate_table: str = Field(description="Filename for the output table")
    intermediate_format: Literal["parquet", "csv", "feather"] = "parquet"
    expected_columns: List[str] = Field(
        default_factory=list,
        description="List of all columns expected in final table"
    )
    expected_row_count_range: Optional[tuple[int, int]] = Field(
        default=None,
        description="Expected min/max row count"
    )


class KeyNormalization(BaseModel):
    column_name: str
    mapping: Dict[str, str] = Field(default_factory=dict)
    format_type: Optional[str] = None
    valid_range: Optional[tuple[Any, Any]] = None


class FileInstruction(BaseModel):
    file_id: str
    original_name: str
    alias: str
    filters: List[str] = Field(default_factory=list)
    rename_columns: Dict[str, str] = Field(default_factory=dict)
    join_keys: List[str] = Field(default_factory=list)
    keep_columns: List[str] = Field(default_factory=list)
    notes: Optional[str] = None


class JoinValidation(BaseModel):
    check_duplicates_on_keys: List[str] = Field(default_factory=list)
    expected_unique: bool = False
    check_row_count_stable: bool = False
    check_no_duplicates_introduced: bool = False
    acceptable_coverage: Optional[float] = None
    max_cardinality_ratio: Optional[float] = None


class JoinStep(BaseModel):
    step: int
    description: str
    left_table: str
    right_table: Optional[str] = None
    join_type: Literal["base", "inner", "left", "right", "outer"] = "base"
    join_keys: List[str] = Field(default_factory=list)
    expected_cardinality: str = "base"
    validation: JoinValidation = Field(default_factory=JoinValidation)


class FeatureEngineering(BaseModel):
    feature_name: str
    description: str
    transform: str
    depends_on: List[str] = Field(default_factory=list)
    implementation: Optional[str] = None


class TimeSplit(BaseModel):
    method: Literal["year-based", "date-based", "rolling-window", "none"] = "none"
    train_years: Optional[str] = None
    test_years: Optional[str] = None
    validation_years: Optional[str] = None
    leakage_check: str = "Not applicable"


class CoverageCheck(BaseModel):
    check: str
    threshold: float
    description: str
    action_if_violation: Optional[str] = None


class CardinalityCheck(BaseModel):
    check: str
    expected: str
    action_if_violation: str


class ValidationSpec(BaseModel):
    time_split: Optional[TimeSplit] = Field(default_factory=lambda: TimeSplit())
    coverage_checks: List[CoverageCheck] = Field(default_factory=list)
    cardinality_checks: List[CardinalityCheck] = Field(default_factory=list)
    additional_checks: List[str] = Field(default_factory=list)


class Stage3Plan(BaseModel):
    # Metadata
    plan_id: str
    selected_task_id: str
    goal: str
    task_category: TaskCategory
    created_at: str = Field(default_factory=lambda: datetime.now().isoformat())
    created_by: str = Field(default="stage3_agent")
    
    # Output specification
    artifacts: ArtifactSpec
    
    # Key normalization
    key_normalization: List[KeyNormalization] = Field(default_factory=list)
    
    # File loading instructions
    file_instructions: List[FileInstruction] = Field(default_factory=list)
    
    # Join strategy
    join_steps: List[JoinStep] = Field(default_factory=list)
    
    # Feature engineering
    feature_engineering: List[FeatureEngineering] = Field(default_factory=list)
    
    # Validation strategy
    validation: ValidationSpec = Field(default_factory=ValidationSpec)
    
    # Expected models and metrics
    expected_model_types: List[str] = Field(default_factory=list)
    evaluation_metrics: List[str] = Field(default_factory=list)
    
    # Documentation
    notes: List[str] = Field(default_factory=list)


# ===========================
# Helper Functions (actual implementations)
# ===========================

def _load_task_proposal_impl(task_id: str) -> str:
    """Load a single TaskProposal by ID."""
    path = STAGE2_OUT_DIR / "task_proposals.json"
    if not path.exists():
        raise FileNotFoundError(f"Could not find task_proposals.json in {STAGE2_OUT_DIR}")

    raw = path.read_text()
    data = json.loads(raw)
    stage2 = Stage2Output.model_validate(data)
    for p in stage2.proposals:
        if p.id == task_id:
            return p.model_dump_json(indent=2)
    raise ValueError(f"No TaskProposal with id={task_id!r} found.")


def _list_data_files_impl() -> List[str]:
    """List available data files."""
    exts = {".csv", ".tsv", ".parquet", ".feather"}
    files = [p.name for p in DATA_DIR.iterdir() if p.suffix.lower() in exts]
    return sorted(files)


def _inspect_data_file_impl(filename: str, n_rows: int = 10) -> str:
    """Inspect a data file - shows head, dtypes, nulls."""
    import pandas as pd

    path = DATA_DIR / filename
    if not path.exists():
        raise FileNotFoundError(f"No such file: {filename}")

    if path.suffix.lower() == ".parquet":
        df = pd.read_parquet(path)
    elif path.suffix.lower() == ".feather":
        df = pd.read_feather(path)
    else:
        df = pd.read_csv(path, nrows=max(n_rows, 100))

    buf = io.StringIO()
    with contextlib.redirect_stdout(buf):
        print(f"=== FILE: {filename} ===")
        print(f"Shape: {df.shape}")
        print(f"\nColumns ({len(df.columns)}):")
        for col in df.columns:
            print(f"  - {col}: {df[col].dtype} (nulls: {df[col].isnull().sum()})")
        print(f"\nFirst {min(n_rows, len(df))} rows:")
        print(df.head(n_rows).to_string())
    return buf.getvalue()


def _python_sandbox_impl(code: str) -> str:
    """Execute Python code for data exploration."""
    import pandas as pd
    
    def load_dataframe(filename: str, n_rows: Optional[int] = None):
        path = DATA_DIR / filename
        if not path.exists():
            raise FileNotFoundError(f"No such file: {filename}")
        if path.suffix.lower() == ".parquet":
            return pd.read_parquet(path)
        elif path.suffix.lower() == ".feather":
            return pd.read_feather(path)
        else:
            if n_rows:
                return pd.read_csv(path, nrows=n_rows)
            return pd.read_csv(path)
    
    globals_dict = {
        "__name__": "__stage3_sandbox__",
        "pd": pd,
        "json": json,
        "Path": Path,
        "DATA_DIR": DATA_DIR,
        "load_dataframe": load_dataframe,
    }
    
    local_env = {}
    buf = io.StringIO()
    try:
        with contextlib.redirect_stdout(buf):
            exec(code, globals_dict, local_env)
    except Exception as e:
        return f"[ERROR] {e}"
    return buf.getvalue() or "[No output]"


def _save_stage3_plan_impl(plan_json: str) -> str:
    """
    Validate and save a Stage3Plan.
    
    Args:
        plan_json: Complete JSON string of Stage3Plan
        
    Returns:
        Success message with path, or raises ValueError
    """
    import pandas as pd
    
    # Parse JSON
    try:
        raw_obj = json.loads(plan_json)
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON: {e}") from e

    # Schema validation
    try:
        plan = Stage3Plan.model_validate(raw_obj)
    except Exception as e:
        raise ValueError(f"Schema validation failed: {e}") from e

    # Validate files exist (use actual function)
    available_files = set(_list_data_files_impl())
    for fi in plan.file_instructions:
        if fi.original_name not in available_files:
            raise ValueError(f"File {fi.original_name!r} not found in DATA_DIR")

    # For join steps, do basic validation
    if plan.join_steps:
        file_cache = {}
        
        # Load files
        for fi in plan.file_instructions:
            try:
                path = DATA_DIR / fi.original_name
                if path.suffix.lower() == ".parquet":
                    df = pd.read_parquet(path)
                else:
                    df = pd.read_csv(path, nrows=100)
                file_cache[fi.alias] = df
            except Exception as e:
                raise ValueError(f"Failed to load {fi.original_name}: {e}") from e
        
        # Validate joins
        for idx, js in enumerate(plan.join_steps):
            if js.join_type == "base":
                if js.left_table not in file_cache:
                    raise ValueError(f"Base table {js.left_table!r} not found")
                continue
                
            if js.left_table not in file_cache:
                raise ValueError(f"Left table {js.left_table!r} not found")
            if js.right_table and js.right_table not in file_cache:
                raise ValueError(f"Right table {js.right_table!r} not found")
            
            # Check keys exist
            df_left = file_cache[js.left_table]
            df_right = file_cache.get(js.right_table) if js.right_table else None
            
            missing_left = [k for k in js.join_keys if k not in df_left.columns]
            if missing_left:
                raise ValueError(f"Join {idx}: keys {missing_left} missing in {js.left_table}")
            
            if df_right is not None:
                missing_right = [k for k in js.join_keys if k not in df_right.columns]
                if missing_right:
                    raise ValueError(f"Join {idx}: keys {missing_right} missing in {js.right_table}")

    # Save
    out_path = STAGE3_OUT_DIR / f"{plan.plan_id}.json"
    out_path.write_text(plan.model_dump_json(indent=2))
    
    return f"✅ Plan saved successfully to: {out_path}"


# ===========================
# Tools (wrapping the implementations)
# ===========================

@tool
def load_task_proposal(task_id: str) -> str:
    """Load a single TaskProposal by ID."""
    return _load_task_proposal_impl(task_id)


@tool
def list_data_files() -> List[str]:
    """List available data files."""
    return _list_data_files_impl()


@tool
def inspect_data_file(filename: str, n_rows: int = 10) -> str:
    """Inspect a data file - shows head, dtypes, nulls."""
    return _inspect_data_file_impl(filename, n_rows)


@tool  
def python_sandbox(code: str) -> str:
    """Execute Python code for data exploration."""
    return _python_sandbox_impl(code)


@tool
def save_stage3_plan(plan_json: str) -> str:
    """
    Validate and save a Stage3Plan.
    
    Args:
        plan_json: Complete JSON string of Stage3Plan
        
    Returns:
        Success message with path, or raises ValueError
    """
    return _save_stage3_plan_impl(plan_json)


TOOLS = [
    load_task_proposal,
    list_data_files,
    inspect_data_file,
    python_sandbox,
    save_stage3_plan,
]

# ===========================
# LLM
# ===========================

llm = ChatOpenAI(
    model="Qwen/Qwen2.5-32B-Instruct",
    base_url="http://127.0.0.1:8001/v1",
    api_key="not-used",
    temperature=0.1,
    max_tokens=8000,
)

llm_with_tools = llm.bind_tools(TOOLS, parallel_tool_calls=False)

# ===========================
# System Prompt - STREAMLINED
# ===========================

STAGE3_SYSTEM_PROMPT = """You are a data pipeline planning agent.

═══════════════════════════════════════════════════════════════
CRITICAL RULES
═══════════════════════════════════════════════════════════════

1. You MUST end by calling save_stage3_plan(plan_json=...) - this is YOUR ONLY SUCCESS CRITERIA
2. NEVER write JSON in your reasoning - build it silently and pass to the tool
3. Be dataset-agnostic - no domain assumptions
4. Set plan_id EXACTLY to "PLAN-{selected_task_id}"

═══════════════════════════════════════════════════════════════
YOUR 3-STEP WORKFLOW
═══════════════════════════════════════════════════════════════

STEP 1: UNDERSTAND (2-3 tool calls)
-----------------------------------
- load_task_proposal(task_id)
- list_data_files()  
- inspect_data_file(filename) for required files

STEP 2: BUILD JSON SILENTLY
----------------------------
In your head, construct the Stage3Plan JSON with these sections:

STEP 2: BUILD JSON SILENTLY
----------------------------
In your head, construct the Stage3Plan JSON with these sections:

```json
{
  "plan_id": "PLAN-{selected_task_id}",  // EXACT format!
  "selected_task_id": "{task_id}",
  "goal": "Brief description",
  "task_category": "descriptive|predictive|unsupervised",
  "artifacts": {
    "intermediate_table": "{task_id}_data.parquet",
    "intermediate_format": "parquet",
    "expected_columns": ["col1", "col2", ...],
    "expected_row_count_range": [min, max]
  },
  "file_instructions": [{
    "file_id": "file1",
    "original_name": "actual_filename.csv",
    "alias": "short_name",
    "rename_columns": {"Original Name": "clean_name"},
    "keep_columns": ["clean_name1", "clean_name2"],
    "filters": [],
    "join_keys": [],
    "notes": null
  }],
  "join_steps": [{
    "step": 1,
    "description": "Load base table",
    "left_table": "short_name",
    "right_table": null,
    "join_type": "base",
    "join_keys": [],
    "expected_cardinality": "base",
    "validation": {}
  }],
  "feature_engineering": [{
    "feature_name": "new_col",
    "description": "What it represents",
    "transform": "mean/sum/etc",
    "depends_on": ["source_col1"],
    "implementation": "df['new_col'] = ..."
  }],
  "validation": {
    "time_split": null,
    "coverage_checks": [],
    "cardinality_checks": [],
    "additional_checks": ["Data loaded", "No duplicates"]
  },
  "expected_model_types": ["Aggregation"],
  "evaluation_metrics": ["Summary Statistics"],
  "notes": ["Any important context"],
  "key_normalization": []
}
```

STEP 3: SAVE (1 tool call - THIS IS MANDATORY)
-----------------------------------------------
Call: save_stage3_plan(plan_json=<your complete JSON as a string>)

═══════════════════════════════════════════════════════════════
EXAMPLES OF CORRECT BEHAVIOR
═══════════════════════════════════════════════════════════════

GOOD (will succeed):
- Round 1: load_task_proposal, list_files, inspect_file
- Round 2: save_stage3_plan(plan_json="...") ✅

BAD (will fail):
- Round 1-4: Never calls save_stage3_plan ❌
- Round 4: Prints JSON in reasoning instead of calling tool ❌

═══════════════════════════════════════════════════════════════
IMPORTANT TIPS
═══════════════════════════════════════════════════════════════

For WIDE FORMAT data (years as columns like "2018-Quantity", "2019-Quantity"):
- Rename to "quantity_2018", "quantity_2019", etc.
- Keep ALL year columns
- Create aggregate features: mean, growth, trend

For SINGLE FILE descriptive tasks:
- join_steps: ONE entry with join_type="base"
- feature_engineering: simple aggregates (mean, sum, growth)
- validation: method="none" for time_split
- expected_model_types: ["Aggregation", "Visualization"]

═══════════════════════════════════════════════════════════════

Remember: Your ONLY job is to call save_stage3_plan() with valid JSON.
DO NOT explain the plan. DO NOT show JSON to the user. JUST SAVE IT."""

# ===========================
# LangGraph
# ===========================


def agent_node(state: MessagesState) -> Dict[str, List[BaseMessage]]:
    """Single LLM step."""
    response: AIMessage = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}


tool_node = ToolNode(TOOLS)


def should_continue(state: MessagesState) -> str:
    last = state["messages"][-1]
    if isinstance(last, AIMessage) and last.tool_calls:
        return "tools"
    return END


builder = StateGraph(MessagesState)
builder.add_node("agent", agent_node)
builder.add_node("tools", tool_node)
builder.set_entry_point("agent")
builder.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
builder.add_edge("tools", "agent")

memory = MemorySaver()
stage3_app = builder.compile(checkpointer=memory)


# ===========================
# Runner
# ===========================


def run_stage3_for_task(task_id: str, max_rounds: int = 15, debug: bool = True):
    """Run Stage 3 agent for a task."""
    
    system_msg = SystemMessage(content=STAGE3_SYSTEM_PROMPT)
    human_msg = HumanMessage(
        content=(
            f"Create Stage 3 plan for '{task_id}'.\n\n"
            f"Steps:\n"
            f"1. load_task_proposal('{task_id}')\n"
            f"2. list_data_files()\n"
            f"3. inspect_data_file() for required files\n"
            f"4. Build Stage3Plan JSON silently\n"
            f"5. save_stage3_plan(plan_json=...) ← MANDATORY FINAL STEP\n\n"
            f"Rules:\n"
            f"- Set plan_id = 'PLAN-{task_id}'\n"
            f"- Never print JSON in reasoning\n"
            f"- Your ONLY success criteria: call save_stage3_plan()\n"
            f"- Complete in 3-4 rounds"
        )
    )

    state: MessagesState = {"messages": [system_msg, human_msg]}

    if not debug:
        return stage3_app.invoke(state, config={"configurable": {"thread_id": f"stage3-{task_id}"}})

    print("=" * 80)
    print(f"🚀 STAGE 3: Planning for {task_id}")
    print("=" * 80)

    final_state = None
    prev_len = 0
    round_num = 0

    for curr_state in stage3_app.stream(
        state,
        config={"configurable": {"thread_id": f"stage3-{task_id}"}},
        stream_mode="values",
    ):
        msgs = curr_state["messages"]
        new_msgs = msgs[prev_len:]

        for m in new_msgs:
            if isinstance(m, SystemMessage):
                print("\n" + "─" * 80)
                print("💻 [SYSTEM]")
                print("─" * 80)
                print(m.content[:500] + "..." if len(m.content) > 500 else m.content)

            elif isinstance(m, HumanMessage):
                print("\n" + "─" * 80)
                print("👤 [USER]")
                print("─" * 80)
                print(m.content)

            elif isinstance(m, AIMessage):
                round_num += 1
                print("\n" + "═" * 80)
                print(f"🤖 [AGENT - Round {round_num}]")
                print("═" * 80)
                
                if m.content:
                    print("\n💭 Reasoning:")
                    content = m.content
                    if len(content) > 1000:
                        print(content[:500] + "\n...[truncated]...\n" + content[-500:])
                    else:
                        print(content)

                if m.tool_calls:
                    print("\n🔧 Tool Calls:")
                    for tc in m.tool_calls:
                        name = getattr(tc, "name", None) or tc.get("name", "UNKNOWN")
                        args = getattr(tc, "args", None) or tc.get("args", {})
                        tc_id = getattr(tc, "id", None) or tc.get("id", "?")
                        
                        print(f"\n  📌 {name}")
                        for k, v in args.items():
                            if isinstance(v, str) and len(v) > 200:
                                print(f"     {k}: {v[:100]}...[truncated]...{v[-100:]}")
                            else:
                                print(f"     {k}: {v}")
                        print(f"     [id: {tc_id}]")

            elif isinstance(m, ToolMessage):
                print("\n" + "─" * 80)
                print(f"📊 [TOOL: {m.name}]")
                print("─" * 80)
                content = str(m.content)
                if len(content) > 1500:
                    lines = content.split('\n')
                    print('\n'.join(lines[:30]) + '\n...[truncated]...\n' + '\n'.join(lines[-10:]))
                else:
                    print(content)

        prev_len = len(msgs)
        final_state = curr_state
        
        # Safety check
        if round_num >= max_rounds:
            print(f"\n⚠️  Reached max rounds ({max_rounds}). Stopping.")
            break

    print("\n" + "=" * 80)
    print(f"✅ Complete - {round_num} rounds")
    print("=" * 80)
    return final_state


# ===========================
# Main
# ===========================

if __name__ == "__main__":
    import sys
    
    if len(sys.argv) < 2:
        print("Usage: python stage3_final_fixed.py <task_id>")
        print("Example: python stage3_final_fixed.py TSK-002")
        sys.exit(1)
    
    task_id = "TSK-002"  # FIXED: strip whitespace
    print(f"\n🎯 Starting Stage 3 for: {task_id}\n")
    
    result = run_stage3_for_task(task_id, debug=True)
    
    # Check for saved plan
    plan_file = STAGE3_OUT_DIR / f"PLAN-{task_id}.json"
    if plan_file.exists():
        print(f"\n✅ SUCCESS! Plan saved to: {plan_file}")
        plan_data = json.loads(plan_file.read_text())
        print("\nPlan Summary:")
        print(f"  Goal: {plan_data.get('goal', 'N/A')}")
        print(f"  Files: {len(plan_data.get('file_instructions', []))}")
        print(f"  Joins: {len(plan_data.get('join_steps', []))}")
        print(f"  Features: {len(plan_data.get('feature_engineering', []))}")
        print(f"  Columns: {len(plan_data.get('artifacts', {}).get('expected_columns', []))}")
    else:
        print("\n⚠️  WARNING: Plan not saved. Check logs above.")

PROJECT_ROOT : /scratch/ziv_baretto/llmserve
DATA_DIR     : /scratch/ziv_baretto/llmserve/data
STAGE2_OUT   : /scratch/ziv_baretto/llmserve/stage2_out
STAGE3_OUT   : /scratch/ziv_baretto/llmserve/stage3_out

🎯 Starting Stage 3 for: TSK-002

🚀 STAGE 3: Planning for TSK-002

────────────────────────────────────────────────────────────────────────────────
💻 [SYSTEM]
────────────────────────────────────────────────────────────────────────────────
You are a data pipeline planning agent.

═══════════════════════════════════════════════════════════════
CRITICAL RULES
═══════════════════════════════════════════════════════════════

1. You MUST end by calling save_stage3_plan(plan_json=...) - this is YOUR ONLY SUCCESS CRITERIA
2. NEVER write JSON in your reasoning - build it silently and pass to the tool
3. Be dataset-agnostic - no domain assumptions
4. Set plan_id EXACTLY to "PLAN-{selected_task_id}"

════════════════════════════════════════...

────────────────────────────────────────────────

Stage 4 Agent

In [3]:
"""
Stage 4: Executor Agent

This agent executes the comprehensive plan created by Stage 3.
It can write and execute its own code/tools to handle any requirement.

Key capabilities:
- Load and parse Stage 3 plans
- Execute data loading, joins, and transformations
- Run models (predictive) or aggregations (descriptive)
- Write custom Python code on the fly
- Validate results and save outputs
- Fully autonomous and dataset-agnostic
"""

from __future__ import annotations

from pathlib import Path
from typing import List, Dict, Any, Optional
from datetime import datetime

import json
import io
import contextlib

from pydantic import BaseModel, Field

from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage, BaseMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END, MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver

# ===========================
# Paths
# ===========================

PROJECT_ROOT = Path("/scratch/ziv_baretto/llmserve").resolve()
DATA_DIR = PROJECT_ROOT / "data"
STAGE3_OUT_DIR = PROJECT_ROOT / "stage3_out"
STAGE4_OUT_DIR = PROJECT_ROOT / "stage4_out"
STAGE4_OUT_DIR.mkdir(parents=True, exist_ok=True)

# Code execution workspace
CODE_WORKSPACE = STAGE4_OUT_DIR / "code_workspace"
CODE_WORKSPACE.mkdir(parents=True, exist_ok=True)

print("PROJECT_ROOT   :", PROJECT_ROOT)
print("DATA_DIR       :", DATA_DIR)
print("STAGE3_OUT_DIR :", STAGE3_OUT_DIR)
print("STAGE4_OUT_DIR :", STAGE4_OUT_DIR)
print("CODE_WORKSPACE :", CODE_WORKSPACE)

# ===========================
# Helper Functions
# ===========================

def _list_stage3_plans_impl() -> List[str]:
    """List all available Stage 3 plans."""
    plans = sorted([p.name for p in STAGE3_OUT_DIR.glob("*.json")])
    return plans


def _load_stage3_plan_impl(plan_id: str) -> str:
    """Load a Stage 3 plan by ID."""
    # Try exact match first
    plan_path = STAGE3_OUT_DIR / f"{plan_id}.json"
    if not plan_path.exists():
        # Try finding by pattern
        matches = list(STAGE3_OUT_DIR.glob(f"*{plan_id}*.json"))
        if not matches:
            raise FileNotFoundError(f"No plan found matching: {plan_id}")
        plan_path = matches[0]
    
    return plan_path.read_text()


def _list_data_files_impl() -> List[str]:
    """List available data files."""
    exts = {".csv", ".tsv", ".parquet", ".feather"}
    files = [p.name for p in DATA_DIR.iterdir() if p.suffix.lower() in exts]
    return sorted(files)


def _save_execution_result_impl(result_json: str) -> str:
    """Save execution results."""
    try:
        result = json.loads(result_json)
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON: {e}")
    
    plan_id = result.get("plan_id", "UNKNOWN")
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    output_path = STAGE4_OUT_DIR / f"execution_{plan_id}_{timestamp}.json"
    output_path.write_text(json.dumps(result, indent=2))
    
    return f"✅ Execution result saved to: {output_path}"


# ===========================
# Tools
# ===========================

@tool
def list_stage3_plans() -> List[str]:
    """List all available Stage 3 plans."""
    return _list_stage3_plans_impl()


@tool
def load_stage3_plan(plan_id: str) -> str:
    """
    Load a Stage 3 plan.
    
    Args:
        plan_id: Plan identifier (e.g., 'PLAN-TSK-002')
    
    Returns:
        JSON string of the plan
    """
    return _load_stage3_plan_impl(plan_id)


@tool
def list_data_files() -> List[str]:
    """List available data files in DATA_DIR."""
    return _list_data_files_impl()


@tool
def execute_python_code(code: str, description: str = "Executing code") -> str:
    """
    Execute arbitrary Python code for data processing, modeling, and analysis.
    
    This is your primary tool for implementing the Stage 3 plan.
    You can:
    - Load and transform data
    - Perform joins and aggregations
    - Build and train models
    - Generate predictions
    - Calculate metrics
    - Save intermediate and final results
    
    Available in the execution environment:
    - pandas as pd
    - numpy as np
    - sklearn (all modules)
    - json, pathlib.Path
    - DATA_DIR, STAGE4_OUT_DIR, CODE_WORKSPACE
    - Helper: load_dataframe(filename) for loading files from DATA_DIR
    
    Args:
        code: Python code to execute
        description: Brief description of what this code does
    
    Returns:
        Output printed to stdout, or error message
    """
    import pandas as pd
    import numpy as np
    from sklearn.linear_model import LinearRegression, Ridge, Lasso
    from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    
    def load_dataframe(filename: str):
        """Load a dataframe from DATA_DIR."""
        path = DATA_DIR / filename
        if not path.exists():
            raise FileNotFoundError(f"File not found: {filename}")
        
        if path.suffix.lower() == ".parquet":
            return pd.read_parquet(path)
        elif path.suffix.lower() == ".feather":
            return pd.read_feather(path)
        else:
            return pd.read_csv(path)
    
    globals_dict = {
        "__name__": "__stage4_executor__",
        "pd": pd,
        "np": np,
        "json": json,
        "Path": Path,
        "DATA_DIR": DATA_DIR,
        "STAGE4_OUT_DIR": STAGE4_OUT_DIR,
        "CODE_WORKSPACE": CODE_WORKSPACE,
        "load_dataframe": load_dataframe,
        "LinearRegression": LinearRegression,
        "Ridge": Ridge,
        "Lasso": Lasso,
        "RandomForestRegressor": RandomForestRegressor,
        "GradientBoostingRegressor": GradientBoostingRegressor,
        "DecisionTreeRegressor": DecisionTreeRegressor,
        "mean_absolute_error": mean_absolute_error,
        "mean_squared_error": mean_squared_error,
        "r2_score": r2_score,
        "train_test_split": train_test_split,
        "StandardScaler": StandardScaler,
    }
    
    local_env = {}
    buf = io.StringIO()
    
    try:
        with contextlib.redirect_stdout(buf):
            print(f"=== {description} ===")
            exec(code, globals_dict, local_env)
    except Exception as e:
        return f"[ERROR] {e}\n\nTraceback: {e.__class__.__name__}"
    
    output = buf.getvalue()
    return output if output else "[Code executed successfully, no output]"


@tool
def save_execution_result(result_json: str) -> str:
    """
    Save the final execution result.
    
    Args:
        result_json: JSON string containing:
            - plan_id: ID of the executed plan
            - task_category: descriptive/predictive/unsupervised
            - status: success/failure/partial
            - outputs: dict with output file paths
            - metrics: dict with performance metrics (if applicable)
            - summary: text summary of results
            - errors: list of any errors encountered
    
    Returns:
        Confirmation message with save path
    """
    return _save_execution_result_impl(result_json)


TOOLS = [
    list_stage3_plans,
    load_stage3_plan,
    list_data_files,
    execute_python_code,
    save_execution_result,
]

# ===========================
# LLM
# ===========================

llm = ChatOpenAI(
    model="Qwen/Qwen2.5-32B-Instruct",
    base_url="http://127.0.0.1:8001/v1",
    api_key="not-used",
    temperature=0.1,
    max_tokens=8000,
)

llm_with_tools = llm.bind_tools(TOOLS, parallel_tool_calls=False)

# ===========================
# System Prompt
# ===========================

STAGE4_SYSTEM_PROMPT = """You are Agent 4: The Executor.

Your mission: Execute the Stage 3 plan flawlessly and autonomously.

═══════════════════════════════════════════════════════════════
CRITICAL RULES
═══════════════════════════════════════════════════════════════

1. You are FULLY AUTONOMOUS - write and execute code to handle ANY requirement
2. You are DATASET-AGNOSTIC - no domain assumptions
3. FOLLOW THE PLAN - the Stage 3 plan is your blueprint
4. VERIFY EVERYTHING - check your work at each step
5. END BY CALLING save_execution_result() - this is your success criterion

═══════════════════════════════════════════════════════════════
YOUR WORKFLOW
═══════════════════════════════════════════════════════════════

PHASE 1: LOAD & UNDERSTAND (2 tool calls)
------------------------------------------
1. list_stage3_plans() - see available plans
2. load_stage3_plan(plan_id) - load the plan you need to execute

Review the plan structure:
- file_instructions: how to load each file
- join_steps: how to combine files
- feature_engineering: derived features to create
- task_category: descriptive/predictive/unsupervised
- expected_model_types: models to try
- evaluation_metrics: how to measure success

PHASE 2: EXECUTE PLAN (multiple execute_python_code calls)
-----------------------------------------------------------
Use execute_python_code() to implement each step. Break into logical chunks:

**Step 1: Load and Transform Data**
```python
# Load files according to file_instructions
# Apply filters
# Rename columns
# Keep only specified columns
```

**Step 2: Perform Joins (if needed)**
```python
# Follow join_steps from the plan
# Validate join results (check row counts, coverage)
```

**Step 3: Feature Engineering**
```python
# Create derived features according to feature_engineering
# Handle missing values appropriately
# Verify feature distributions
```

**Step 4: Execute Analysis**

For DESCRIPTIVE tasks:
```python
# Calculate aggregations (mean, sum, count, etc.)
# Generate summary statistics
# Create pivot tables or groupby results
# Save summary tables to STAGE4_OUT_DIR
```

For PREDICTIVE tasks:
```python
# Split data (train/test according to validation.time_split)
# Train models (try each from expected_model_types)
# Generate predictions
# Calculate metrics (from evaluation_metrics)
# Save predictions and model metrics
```

**Step 5: Save Results**
```python
# Save intermediate data
# Save final outputs (predictions, summaries, etc.)
# Log all output paths
```

PHASE 3: REPORT RESULTS (1 tool call)
--------------------------------------
Call save_execution_result() with a JSON containing:
```json
{
  "plan_id": "PLAN-TSK-002",
  "task_category": "descriptive",
  "status": "success",
  "outputs": {
    "intermediate_data": "path/to/data.parquet",
    "summary_statistics": "path/to/summary.csv",
    "aggregations": "path/to/aggregations.csv"
  },
  "metrics": {
    "rows_processed": 1000,
    "features_created": 5
  },
  "summary": "Successfully analyzed export trends. Created 5 aggregate features...",
  "errors": []
}
```

═══════════════════════════════════════════════════════════════
GUIDELINES FOR WRITING CODE
═══════════════════════════════════════════════════════════════

**Be Incremental**: Break complex operations into smaller execute_python_code() calls
- First call: Load and inspect data
- Second call: Perform transformations
- Third call: Apply joins
- Fourth call: Feature engineering
- Fifth call: Run analysis
- Sixth call: Save results

**Be Defensive**: Add checks and validations
```python
# Check data loaded correctly
print(f"Loaded {len(df)} rows, {len(df.columns)} columns")

# Validate joins
print(f"After join: {len(merged)} rows (started with {len(df_left)})")

# Check for nulls
print(f"Null counts:\n{df.isnull().sum()}")
```

**Be Clear**: Print progress and intermediate results
```python
print("Step 1: Loading data...")
print("Step 2: Applying transformations...")
print("Step 3: Creating features...")
```

**Handle Wide Format Data**: If plan renames columns like "2018-Quantity" to "quantity_2018":
```python
# For creating aggregates across year columns
year_cols = [c for c in df.columns if c.startswith('quantity_')]
df['quantity_mean'] = df[year_cols].mean(axis=1)
df['quantity_total'] = df[year_cols].sum(axis=1)
```

**Save Intermediate Results**: Don't lose your work
```python
# Save intermediate data
output_path = STAGE4_OUT_DIR / "intermediate_data.parquet"
df.to_parquet(output_path, index=False)
print(f"Saved to: {output_path}")
```

═══════════════════════════════════════════════════════════════
EXAMPLE: DESCRIPTIVE TASK EXECUTION
═══════════════════════════════════════════════════════════════

Plan says:
- Task: Analyze export trends
- File: "exports.csv" with year columns
- Features: Create mean, growth rate
- Type: Descriptive

Your execution:
1. execute_python_code: Load file, apply renames, inspect
2. execute_python_code: Create aggregate features (mean, growth)
3. execute_python_code: Calculate summary statistics, save
4. save_execution_result: Report success with output paths

═══════════════════════════════════════════════════════════════
EXAMPLE: PREDICTIVE TASK EXECUTION
═══════════════════════════════════════════════════════════════

Plan says:
- Task: Predict yield
- Files: main.csv, features.csv
- Join: on ["state", "year"]
- Features: lag_yield, rainfall_ratio
- Models: RandomForest, XGBoost
- Train: 2000-2020, Test: 2021-2024

Your execution:
1. execute_python_code: Load both files
2. execute_python_code: Perform join, validate
3. execute_python_code: Create features (lag, ratio)
4. execute_python_code: Split train/test by year
5. execute_python_code: Train RandomForest, calculate metrics
6. execute_python_code: Save predictions and metrics
7. save_execution_result: Report success with metrics

═══════════════════════════════════════════════════════════════
TROUBLESHOOTING
═══════════════════════════════════════════════════════════════

**If a join fails**: Check column names match exactly, inspect unique values
**If feature creation fails**: Print dtypes, check for nulls, verify column names
**If model training fails**: Check for infinite/NaN values, verify train/test split
**If file save fails**: Ensure directory exists, check permissions

You can ALWAYS write more code to investigate and fix issues!

═══════════════════════════════════════════════════════════════

Remember:
- You are autonomous - handle ANY issue that arises
- Follow the Stage 3 plan - it's your blueprint
- Write clean, incremental code
- Verify your work at each step
- End by calling save_execution_result()

Your success = Executing the plan and saving results."""

# ===========================
# LangGraph
# ===========================

def agent_node(state: MessagesState) -> Dict[str, List[BaseMessage]]:
    """LLM agent step."""
    response: AIMessage = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}


tool_node = ToolNode(TOOLS)


def should_continue(state: MessagesState) -> str:
    """Route based on tool calls."""
    last = state["messages"][-1]
    if isinstance(last, AIMessage) and last.tool_calls:
        return "tools"
    return END


builder = StateGraph(MessagesState)
builder.add_node("agent", agent_node)
builder.add_node("tools", tool_node)
builder.set_entry_point("agent")
builder.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
builder.add_edge("tools", "agent")

memory = MemorySaver()
stage4_app = builder.compile(checkpointer=memory)


# ===========================
# Runner
# ===========================

def run_stage4_for_plan(plan_id: str, max_rounds: int = 25, debug: bool = True):
    """Execute a Stage 3 plan."""
    
    system_msg = SystemMessage(content=STAGE4_SYSTEM_PROMPT)
    human_msg = HumanMessage(
        content=(
            f"Execute Stage 3 plan: '{plan_id}'\n\n"
            f"Workflow:\n"
            f"1. load_stage3_plan('{plan_id}')\n"
            f"2. Understand the plan structure\n"
            f"3. Execute each step using execute_python_code()\n"
            f"4. Verify results at each stage\n"
            f"5. save_execution_result() with final outputs\n\n"
            f"Be autonomous. Handle any issues. Follow the plan.\n"
            f"Your success = Plan executed + Results saved."
        )
    )

    state: MessagesState = {"messages": [system_msg, human_msg]}

    if not debug:
        return stage4_app.invoke(state, config={"configurable": {"thread_id": f"stage4-{plan_id}"}})

    print("=" * 80)
    print(f"🚀 STAGE 4: Executing plan {plan_id}")
    print("=" * 80)

    final_state = None
    prev_len = 0
    round_num = 0

    for curr_state in stage4_app.stream(
        state,
        config={"configurable": {"thread_id": f"stage4-{plan_id}"}},
        stream_mode="values",
    ):
        msgs = curr_state["messages"]
        new_msgs = msgs[prev_len:]

        for m in new_msgs:
            if isinstance(m, SystemMessage):
                print("\n" + "─" * 80)
                print("💻 [SYSTEM]")
                print("─" * 80)
                print(m.content[:500] + "..." if len(m.content) > 500 else m.content)

            elif isinstance(m, HumanMessage):
                print("\n" + "─" * 80)
                print("👤 [USER]")
                print("─" * 80)
                print(m.content)

            elif isinstance(m, AIMessage):
                round_num += 1
                print("\n" + "═" * 80)
                print(f"🤖 [AGENT - Round {round_num}]")
                print("═" * 80)
                
                if m.content:
                    print("\n💭 Reasoning:")
                    content = m.content
                    if len(content) > 1000:
                        print(content[:500] + "\n...[truncated]...\n" + content[-500:])
                    else:
                        print(content)

                if m.tool_calls:
                    print("\n🔧 Tool Calls:")
                    for tc in m.tool_calls:
                        name = getattr(tc, "name", tc.get("name", "UNKNOWN"))
                        args = getattr(tc, "args", tc.get("args", {}))
                        
                        print(f"\n  📌 {name}")
                        for k, v in args.items():
                            if isinstance(v, str) and len(v) > 300:
                                print(f"     {k}: {v[:150]}...[truncated]...{v[-150:]}")
                            else:
                                print(f"     {k}: {v}")

            elif isinstance(m, ToolMessage):
                print("\n" + "─" * 80)
                print(f"📊 [TOOL: {m.name}]")
                print("─" * 80)
                content = str(m.content)
                if len(content) > 2000:
                    lines = content.split('\n')
                    print('\n'.join(lines[:40]) + '\n...[truncated]...\n' + '\n'.join(lines[-15:]))
                else:
                    print(content)

        prev_len = len(msgs)
        final_state = curr_state
        
        if round_num >= max_rounds:
            print(f"\n⚠️  Reached max rounds ({max_rounds})")
            break

    print("\n" + "=" * 80)
    print(f"✅ Complete - {round_num} rounds")
    print("=" * 80)
    return final_state


# ===========================
# Main
# ===========================

if __name__ == "__main__":
    import sys
    
    if len(sys.argv) < 2:
        print("Usage: python stage4_executor.py <plan_id>")
        print("Example: python stage4_executor.py PLAN-TSK-002")
        print("\nAvailable plans:")
        for plan in _list_stage3_plans_impl():
            print(f"  - {plan}")
        sys.exit(1)
    
    plan_id = "PLAN-TSK-002"
    print(f"\n🎯 Executing plan: {plan_id}\n")
    
    result = run_stage4_for_plan(plan_id, debug=True)
    
    # Check for execution results
    results = sorted(STAGE4_OUT_DIR.glob(f"execution_{plan_id}*.json"))
    if results:
        latest = results[-1]
        print(f"\n✅ Execution result: {latest}")
        result_data = json.loads(latest.read_text())
        print("\nSummary:")
        print(f"  Status: {result_data.get('status', 'N/A')}")
        print(f"  Outputs: {len(result_data.get('outputs', {}))}")
        if result_data.get('metrics'):
            print(f"  Metrics: {result_data['metrics']}")
    else:
        print("\n⚠️  No execution result saved")

PROJECT_ROOT   : /scratch/ziv_baretto/llmserve
DATA_DIR       : /scratch/ziv_baretto/llmserve/data
STAGE3_OUT_DIR : /scratch/ziv_baretto/llmserve/stage3_out
STAGE4_OUT_DIR : /scratch/ziv_baretto/llmserve/stage4_out
CODE_WORKSPACE : /scratch/ziv_baretto/llmserve/stage4_out/code_workspace

🎯 Executing plan: PLAN-TSK-002

🚀 STAGE 4: Executing plan PLAN-TSK-002

────────────────────────────────────────────────────────────────────────────────
💻 [SYSTEM]
────────────────────────────────────────────────────────────────────────────────
You are Agent 4: The Executor.

Your mission: Execute the Stage 3 plan flawlessly and autonomously.

═══════════════════════════════════════════════════════════════
CRITICAL RULES
═══════════════════════════════════════════════════════════════

1. You are FULLY AUTONOMOUS - write and execute code to handle ANY requirement
2. You are DATASET-AGNOSTIC - no domain assumptions
3. FOLLOW THE PLAN - the Stage 3 plan is your blueprint
4. VERIFY EVERYTHING - check your 

Stage 5

In [9]:
"""
Stage 5: Visualizer Agent

This agent creates comprehensive visualizations and reports from Stage 4 execution results.
It can write and execute its own visualization code to handle any requirement.

Key capabilities:
- Load Stage 4 execution results
- Create appropriate visualizations based on task type
- Generate comprehensive HTML reports
- Write custom visualization code on the fly
- Fully autonomous and dataset-agnostic
"""

from __future__ import annotations

from pathlib import Path
from typing import List, Dict, Any, Optional
from datetime import datetime

import json
import io
import contextlib

from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage, BaseMessage, messages_to_dict
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END, MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver

# ===========================
# Paths
# ===========================

PROJECT_ROOT = Path("/scratch/ziv_baretto/llmserve").resolve()
DATA_DIR = PROJECT_ROOT / "data"
STAGE3_OUT_DIR = PROJECT_ROOT / "stage3_out"
STAGE4_OUT_DIR = PROJECT_ROOT / "stage4_out"
STAGE5_OUT_DIR = PROJECT_ROOT / "stage5_out"
STAGE5_OUT_DIR.mkdir(parents=True, exist_ok=True)

# Visualization workspace
VIZ_WORKSPACE = STAGE5_OUT_DIR / "viz_workspace"
VIZ_WORKSPACE.mkdir(parents=True, exist_ok=True)

print("PROJECT_ROOT   :", PROJECT_ROOT)
print("STAGE4_OUT_DIR :", STAGE4_OUT_DIR)
print("STAGE5_OUT_DIR :", STAGE5_OUT_DIR)
print("VIZ_WORKSPACE  :", VIZ_WORKSPACE)

# ===========================
# Helper Functions
# ===========================

def _list_stage4_results_impl() -> List[str]:
    """List all available Stage 4 execution results."""
    results = sorted([p.name for p in STAGE4_OUT_DIR.glob("execution_*.json")])
    return results


def _load_stage4_result_impl(result_id: str) -> str:
    """Load a Stage 4 execution result."""
    # Try exact match first
    result_path = STAGE4_OUT_DIR / result_id
    if not result_path.exists():
        # Try finding by pattern
        matches = list(STAGE4_OUT_DIR.glob(f"*{result_id}*.json"))
        if not matches:
            raise FileNotFoundError(f"No result found matching: {result_id}")
        result_path = matches[0]
    
    return result_path.read_text()


def _load_stage3_plan_impl(plan_id: str) -> str:
    """Load a Stage 3 plan."""
    plan_path = STAGE3_OUT_DIR / f"{plan_id}.json"
    if not plan_path.exists():
        matches = list(STAGE3_OUT_DIR.glob(f"*{plan_id}*.json"))
        if not matches:
            raise FileNotFoundError(f"No plan found matching: {plan_id}")
        plan_path = matches[0]
    
    return plan_path.read_text()


def _save_visualization_report_impl(report_json: str) -> str:
    """Save visualization report."""
    try:
        report = json.loads(report_json)
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON: {e}")
    
    plan_id = report.get("plan_id", "UNKNOWN")
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    output_path = STAGE5_OUT_DIR / f"visualization_report_{plan_id}_{timestamp}.json"
    output_path.write_text(json.dumps(report, indent=2))
    
    return f"✅ Visualization report saved to: {output_path}"


# ===========================
# Tools
# ===========================

@tool
def list_stage4_results() -> List[str]:
    """List all available Stage 4 execution results."""
    return _list_stage4_results_impl()


@tool
def load_stage4_result(result_id: str) -> str:
    """
    Load a Stage 4 execution result.
    
    Args:
        result_id: Result filename or pattern
    
    Returns:
        JSON string of the result
    """
    return _load_stage4_result_impl(result_id)


@tool
def load_stage3_plan(plan_id: str) -> str:
    """
    Load a Stage 3 plan for context.
    
    Args:
        plan_id: Plan identifier
    
    Returns:
        JSON string of the plan
    """
    return _load_stage3_plan_impl(plan_id)


@tool
def create_visualizations(code: str, description: str = "Creating visualizations") -> str:
    """
    Execute Python code to create visualizations and reports.
    
    Use this to:
    - Load data from Stage 4 outputs
    - Create plots (matplotlib, seaborn, plotly)
    - Generate summary tables
    - Create HTML reports
    - Save visualizations to STAGE5_OUT_DIR
    
    Available in the environment:
    - pandas as pd
    - numpy as np
    - matplotlib.pyplot as plt
    - seaborn as sns
    - plotly.express as px
    - plotly.graph_objects as go
    - json, pathlib.Path
    - STAGE4_OUT_DIR, STAGE5_OUT_DIR, VIZ_WORKSPACE
    
    Args:
        code: Python code to execute
        description: Brief description of what this code creates
    
    Returns:
        Output printed to stdout, or error message
    """
    import pandas as pd
    import numpy as np
    import matplotlib
    matplotlib.use('Agg')  # Non-interactive backend
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    # Try to import plotly (optional)
    try:
        import plotly.express as px
        import plotly.graph_objects as go
        has_plotly = True
    except ImportError:
        px = None
        go = None
        has_plotly = False
    
    def load_dataframe(filepath):
        """Load a dataframe from any supported format."""
        filepath = Path(filepath)
        if not filepath.exists():
            # Try relative to STAGE4_OUT_DIR
            filepath = STAGE4_OUT_DIR / filepath.name
        
        if not filepath.exists():
            raise FileNotFoundError(f"File not found: {filepath}")
        
        if filepath.suffix.lower() == ".parquet":
            return pd.read_parquet(filepath)
        elif filepath.suffix.lower() == ".feather":
            return pd.read_feather(filepath)
        elif filepath.suffix.lower() in [".csv", ".tsv"]:
            return pd.read_csv(filepath)
        else:
            raise ValueError(f"Unsupported format: {filepath.suffix}")
    
    globals_dict = {
        "__name__": "__stage5_visualizer__",
        "pd": pd,
        "np": np,
        "plt": plt,
        "sns": sns,
        "json": json,
        "Path": Path,
        "STAGE4_OUT_DIR": STAGE4_OUT_DIR,
        "STAGE5_OUT_DIR": STAGE5_OUT_DIR,
        "VIZ_WORKSPACE": VIZ_WORKSPACE,
        "load_dataframe": load_dataframe,
    }
    
    if has_plotly:
        globals_dict.update({"px": px, "go": go})
    
    # Use globals_dict for both globals and locals to avoid scope issues
    buf = io.StringIO()
    
    try:
        with contextlib.redirect_stdout(buf):
            print(f"=== {description} ===")
            # Use same dict for both to keep variables in scope
            exec(code, globals_dict, globals_dict)
            # Close any open plots
            plt.close('all')
    except Exception as e:
        plt.close('all')
        import traceback
        error_details = traceback.format_exc()
        return f"[ERROR] {e}\n\n{error_details}"
    
    output = buf.getvalue()
    return output if output else "[Visualizations created successfully]"


@tool
def save_visualization_report(report_json: str) -> str:
    """
    Save the final visualization report.
    
    Args:
        report_json: JSON string containing:
            - plan_id: ID of the executed plan
            - task_category: descriptive/predictive/unsupervised
            - visualizations: list of created visualization paths
            - html_report: path to HTML report (if created)
            - summary: text summary of visualizations
            - insights: key insights from the visualizations
    
    Returns:
        Confirmation message with save path
    """
    return _save_visualization_report_impl(report_json)


TOOLS = [
    list_stage4_results,
    load_stage4_result,
    load_stage3_plan,
    create_visualizations,
    save_visualization_report,
]

# ===========================
# LLM
# ===========================

llm = ChatOpenAI(
    model="Qwen/Qwen2.5-32B-Instruct",
    base_url="http://127.0.0.1:8001/v1",
    api_key="not-used",
    temperature=0.2,
    max_tokens=8000,
)

llm_with_tools = llm.bind_tools(TOOLS, parallel_tool_calls=False)

# ===========================
# System Prompt
# ===========================

STAGE5_SYSTEM_PROMPT = """You are Agent 5: The Visualizer.

Your mission: Create comprehensive, insightful visualizations from Stage 4 execution results.

═══════════════════════════════════════════════════════════════
CRITICAL RULES
═══════════════════════════════════════════════════════════════

1. You are FULLY AUTONOMOUS - write visualization code for ANY data type
2. You are DATASET-AGNOSTIC - adapt to any domain
3. CREATE PUBLICATION-QUALITY VISUALS - clear, informative, professional
4. TELL A STORY - your visualizations should reveal insights
5. END BY CALLING save_visualization_report() - this is your success criterion

═══════════════════════════════════════════════════════════════
YOUR WORKFLOW
═══════════════════════════════════════════════════════════════

PHASE 1: LOAD & UNDERSTAND (2-3 tool calls)
--------------------------------------------
1. list_stage4_results() - see available execution results
2. load_stage4_result(result_id) - load the result you'll visualize
3. load_stage3_plan(plan_id) - understand the original task context

Review what you have:
- Task category (descriptive/predictive/unsupervised)
- Output files from Stage 4
- Metrics and summaries
- Original problem statement

PHASE 2: CREATE VISUALIZATIONS (multiple create_visualizations calls)
----------------------------------------------------------------------
**CRITICAL: Always inspect data structure FIRST before creating plots!**

⚠️  NEVER assume column names - ALWAYS inspect first! ⚠️
⚠️  CREATE SEPARATE PNG FILES - one chart per file! ⚠️
⚠️  USE CLEAR LABELS - titles, axes, units, readable fonts! ⚠️

**Your first call to create_visualizations() MUST be for inspection:**
- Load the data file
- Print df.columns
- Print df.head()
- Print df.dtypes

**After inspection, create SEPARATE visualizations:**
- Each chart = ONE PNG file (not subplots!)
- Use plt.figure() for each chart, then plt.close()
- Use actual category/variety names, not indices
- Large fonts: titles 16pt, axes 14pt
- Include units in axis labels
- High resolution: dpi=300

Use create_visualizations() to make compelling visuals. Break into logical chunks:

**Step 1: INSPECT the data structure**
```python
# Load and inspect
df = load_dataframe("path/to/file.csv")
print("Shape:", df.shape)
print("Columns:", list(df.columns))
print("First few rows:")
print(df.head())
print("\nData types:")
print(df.dtypes)
```

**Step 2: Create visualizations based on ACTUAL columns**

For DESCRIPTIVE tasks:

**Call 1: INSPECT data structure FIRST**
```python
# CRITICAL: Always inspect before plotting!
df = load_dataframe("summary_statistics.csv")
print("=== DATA INSPECTION ===")
print(f"Shape: {df.shape}")
print(f"\nColumn names:")
for i, col in enumerate(df.columns):
    print(f"  {i}: {col}")
print(f"\nFirst 3 rows:")
print(df.head(3))
print(f"\nColumn data types:")
print(df.dtypes)
```

**Call 2: Create SEPARATE visualizations with CLEAR labels**

🎨 **CRITICAL VISUALIZATION RULES:**
1. **Create SEPARATE PNG files** - one chart per file, not subplots
2. **Use CLEAR, DESCRIPTIVE axis labels** with units
3. **Use ACTUAL category names** - not indices (0, 1, 2...)
4. **Large, readable fonts** - titles 16pt, labels 14pt
5. **One main insight per chart** - don't cram multiple things together

```python
# Load data
df = load_dataframe("summary_statistics.csv")

# Chart 1: Quantity trend over time (SEPARATE FILE)
plt.figure(figsize=(12, 6))
quantity_cols = sorted([c for c in df.columns if 'quantity' in c.lower() and '_20' in c])

# Extract years properly (e.g., 'quantity_2018_19' -> '2018-19')
years = []
for col in quantity_cols:
    # Extract the year part after 'quantity_'
    year_part = col.split('quantity_')[-1]
    years.append(year_part.replace('_', '-'))

totals = df[quantity_cols].sum().values

plt.plot(years, totals, marker='o', linewidth=3, markersize=10, color='#1f77b4')
plt.title('Rice Export Quantity Trend Over Time', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Year', fontsize=14, fontweight='bold')
plt.ylabel('Total Quantity (Tonnes)', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, alpha=0.3, linestyle='--')
plt.tight_layout()
plt.savefig(STAGE5_OUT_DIR / 'quantity_trend.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Saved: quantity_trend.png")

# Chart 2: Total quantity by rice variety (SEPARATE FILE)
plt.figure(figsize=(12, 6))

# Get actual category names if available, otherwise use row data
if 'description' in df.columns or 'Description' in df.columns:
    desc_col = 'description' if 'description' in df.columns else 'Description'
    categories = df[desc_col].tolist()
else:
    # Use first column or create meaningful names
    categories = [f"Category {i+1}" for i in range(len(df))]

# Use total_quantity if available
if 'total_quantity' in df.columns:
    values = df['total_quantity'].values
else:
    # Sum all quantity columns
    quantity_cols = [c for c in df.columns if 'quantity' in c.lower() and '_20' in c]
    values = df[quantity_cols].sum(axis=1).values

plt.barh(categories, values, color='#ff7f0e')
plt.title('Total Rice Export Quantity by Variety', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Total Quantity (Tonnes)', fontsize=14, fontweight='bold')
plt.ylabel('Rice Variety', fontsize=14, fontweight='bold')
plt.xticks(fontsize=12)
plt.yticks(fontsize=11)
plt.grid(True, alpha=0.3, axis='x', linestyle='--')
plt.tight_layout()
plt.savefig(STAGE5_OUT_DIR / 'quantity_by_variety.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Saved: quantity_by_variety.png")

# Chart 3: Value trend over time (INR) (SEPARATE FILE)
plt.figure(figsize=(12, 6))
value_cols = sorted([c for c in df.columns if 'value_INR' in c or 'value_inr' in c])

# Extract years
years = []
for col in value_cols:
    year_part = col.split('_')[-2] + '-' + col.split('_')[-1]  # Get year part
    years.append(year_part)

totals = df[value_cols].sum().values / 1e6  # Convert to millions

plt.plot(years, totals, marker='s', linewidth=3, markersize=10, color='#2ca02c')
plt.title('Rice Export Value Trend Over Time (INR)', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Year', fontsize=14, fontweight='bold')
plt.ylabel('Total Value (Million INR)', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, alpha=0.3, linestyle='--')
plt.tight_layout()
plt.savefig(STAGE5_OUT_DIR / 'value_trend_inr.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Saved: value_trend_inr.png")

# Chart 4: Comparison of varieties (SEPARATE FILE)
plt.figure(figsize=(14, 8))
if 'total_quantity' in df.columns and 'total_value_INR' in df.columns:
    # Create a nice comparison chart
    x = range(len(categories))
    width = 0.35
    
    fig, ax1 = plt.subplots(figsize=(14, 8))
    
    # Quantity bars
    ax1.bar([i - width/2 for i in x], df['total_quantity'], width, 
            label='Quantity (Tonnes)', color='#1f77b4', alpha=0.8)
    ax1.set_xlabel('Rice Variety', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Quantity (Tonnes)', fontsize=14, fontweight='bold', color='#1f77b4')
    ax1.tick_params(axis='y', labelcolor='#1f77b4', labelsize=12)
    ax1.set_xticks(x)
    ax1.set_xticklabels(categories, rotation=45, ha='right', fontsize=11)
    
    # Value on secondary axis
    ax2 = ax1.twinx()
    ax2.bar([i + width/2 for i in x], df['total_value_INR']/1e6, width,
            label='Value (Million INR)', color='#ff7f0e', alpha=0.8)
    ax2.set_ylabel('Value (Million INR)', fontsize=14, fontweight='bold', color='#ff7f0e')
    ax2.tick_params(axis='y', labelcolor='#ff7f0e', labelsize=12)
    
    plt.title('Rice Exports: Quantity vs Value by Variety', fontsize=16, fontweight='bold', pad=20)
    
    # Legends
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=12)
    
    plt.tight_layout()
    plt.savefig(STAGE5_OUT_DIR / 'quantity_vs_value.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✅ Saved: quantity_vs_value.png")

print("\n✅ Created 4 separate, clean visualizations!")
```

Visualization 2: More detailed charts (SEPARATE FILES)
```python
# Each chart should be its own file!

# Chart: Top exporters
plt.figure(figsize=(10, 6))
# ... your visualization code
plt.savefig(STAGE5_OUT_DIR / 'top_exporters.png', dpi=300, bbox_inches='tight')
plt.close()

# Chart: Year-over-year growth
plt.figure(figsize=(10, 6))  
# ... your visualization code
plt.savefig(STAGE5_OUT_DIR / 'yoy_growth.png', dpi=300, bbox_inches='tight')
plt.close()
```

**Key principles:**
- ✅ ONE chart = ONE PNG file
- ✅ Clear, descriptive titles (16pt bold)
- ✅ Labeled axes with units (14pt bold)
- ✅ Use actual category names, not indices
- ✅ Large figure size (12x6 or 14x8)
- ✅ High DPI (300)
- ❌ Don't use subplots (axes[0,0], axes[0,1])
- ❌ Don't use numeric indices for categories
- ❌ Don't cram multiple things in one image

**For PREDICTIVE tasks:**

**Create SEPARATE visualizations for model performance:**

```python
# Load predictions
df = load_dataframe("predictions.csv")

# Chart 1: Actual vs Predicted (SEPARATE FILE)
plt.figure(figsize=(10, 10))
plt.scatter(df['actual'], df['predicted'], alpha=0.6, s=50, color='#1f77b4')
plt.plot([df['actual'].min(), df['actual'].max()], 
         [df['actual'].min(), df['actual'].max()], 
         'r--', linewidth=2, label='Perfect Prediction')
plt.xlabel('Actual Values', fontsize=14, fontweight='bold')
plt.ylabel('Predicted Values', fontsize=14, fontweight='bold')
plt.title('Model Performance: Actual vs Predicted', fontsize=16, fontweight='bold', pad=20)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(STAGE5_OUT_DIR / 'actual_vs_predicted.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Saved: actual_vs_predicted.png")

# Chart 2: Residuals (SEPARATE FILE)
plt.figure(figsize=(12, 6))
residuals = df['actual'] - df['predicted']
plt.scatter(df['predicted'], residuals, alpha=0.6, s=50, color='#ff7f0e')
plt.axhline(y=0, color='r', linestyle='--', linewidth=2)
plt.xlabel('Predicted Values', fontsize=14, fontweight='bold')
plt.ylabel('Residuals', fontsize=14, fontweight='bold')
plt.title('Residual Plot', fontsize=16, fontweight='bold', pad=20)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(STAGE5_OUT_DIR / 'residuals.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Saved: residuals.png")

# Chart 3: Error distribution (SEPARATE FILE)
plt.figure(figsize=(12, 6))
plt.hist(residuals, bins=30, edgecolor='black', color='#2ca02c', alpha=0.7)
plt.xlabel('Prediction Error', fontsize=14, fontweight='bold')
plt.ylabel('Frequency', fontsize=14, fontweight='bold')
plt.title('Distribution of Prediction Errors', fontsize=16, fontweight='bold', pad=20)
plt.axvline(x=0, color='r', linestyle='--', linewidth=2)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig(STAGE5_OUT_DIR / 'error_distribution.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Saved: error_distribution.png")

# Chart 4: Time series (if date column exists)
if 'date' in df.columns or 'year' in df.columns:
    plt.figure(figsize=(14, 6))
    time_col = 'date' if 'date' in df.columns else 'year'
    plt.plot(df[time_col], df['actual'], marker='o', label='Actual', linewidth=2)
    plt.plot(df[time_col], df['predicted'], marker='s', label='Predicted', linewidth=2)
    plt.xlabel('Time', fontsize=14, fontweight='bold')
    plt.ylabel('Value', fontsize=14, fontweight='bold')
    plt.title('Predictions Over Time', fontsize=16, fontweight='bold', pad=20)
    plt.legend(fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(STAGE5_OUT_DIR / 'predictions_over_time.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✅ Saved: predictions_over_time.png")
```

**Feature Importance (SEPARATE FILE):**
```python
# If feature importance available, visualize it
# Create segment-wise analysis plots
```

**HTML Report (references all your separate PNG files):**
```python
# After creating all your separate PNG files, create the HTML report

html_content = '''
<!DOCTYPE html>
<html>
<head>
    <title>Rice Export Analysis Report</title>
    <style>
        body { 
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; 
            margin: 40px; 
            background-color: #f5f5f5;
        }
        .container {
            max-width: 1200px;
            margin: 0 auto;
            background-color: white;
            padding: 40px;
            box-shadow: 0 0 10px rgba(0,0,0,0.1);
        }
        h1 { 
            color: #2c3e50; 
            border-bottom: 3px solid #3498db;
            padding-bottom: 10px;
        }
        h2 { 
            color: #34495e; 
            margin-top: 40px; 
            border-left: 4px solid #3498db;
            padding-left: 15px;
        }
        img { 
            max-width: 100%; 
            height: auto; 
            margin: 20px 0; 
            border: 1px solid #ddd;
            border-radius: 4px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }
        .summary { 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
            color: white;
            padding: 30px; 
            border-radius: 8px;
            margin: 20px 0;
        }
        .metric { 
            display: inline-block; 
            margin: 15px 30px; 
            text-align: center;
        }
        .metric-value { 
            font-size: 32px; 
            font-weight: bold; 
            display: block;
            margin-bottom: 5px;
        }
        .metric-label {
            font-size: 14px;
            opacity: 0.9;
        }
        .insight {
            background: #e8f4f8;
            border-left: 4px solid #3498db;
            padding: 15px;
            margin: 10px 0;
            border-radius: 4px;
        }
        .chart-section {
            margin: 40px 0;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Rice Export Analysis Report</h1>
        
        <div class="summary">
            <h2 style="color: white; border: none; margin-top: 0;">Executive Summary</h2>
            <p><strong>Task:</strong> Analysis of rice export trends from 2018-2025</p>
            <div class="metric">
                <span class="metric-value">{rows_analyzed}</span>
                <span class="metric-label">Rice Varieties Analyzed</span>
            </div>
            <div class="metric">
                <span class="metric-value">7</span>
                <span class="metric-label">Years of Data</span>
            </div>
            <div class="metric">
                <span class="metric-value">4</span>
                <span class="metric-label">Visualizations Created</span>
            </div>
        </div>
        
        <h2>📈 Quantity Trends</h2>
        <div class="chart-section">
            <img src="quantity_trend.png" alt="Quantity Trend Over Time">
            <p><em>Figure 1: Total rice export quantities showing year-over-year trends</em></p>
        </div>
        
        <div class="chart-section">
            <img src="quantity_by_variety.png" alt="Quantity by Variety">
            <p><em>Figure 2: Total export quantities broken down by rice variety</em></p>
        </div>
        
        <h2>💰 Value Trends</h2>
        <div class="chart-section">
            <img src="value_trend_inr.png" alt="Value Trend">
            <p><em>Figure 3: Export value trends in Indian Rupees (INR)</em></p>
        </div>
        
        <h2>🔍 Comparative Analysis</h2>
        <div class="chart-section">
            <img src="quantity_vs_value.png" alt="Quantity vs Value">
            <p><em>Figure 4: Comparison of quantity and value across varieties</em></p>
        </div>
        
        <h2>💡 Key Insights</h2>
        <div class="insight">
            <strong>📊 Insight 1:</strong> Export quantities show significant variation across years, with a notable peak in 2024-25.
        </div>
        <div class="insight">
            <strong>📊 Insight 2:</strong> Certain rice varieties dominate both quantity and value metrics.
        </div>
        <div class="insight">
            <strong>📊 Insight 3:</strong> Value trends don't always correlate directly with quantity, suggesting price variations.
        </div>
        
        <h2>📋 Recommendations</h2>
        <ul>
            <li>Focus on high-value varieties for maximum revenue</li>
            <li>Investigate factors behind quantity fluctuations</li>
            <li>Monitor price trends for strategic planning</li>
        </ul>
    </div>
</body>
</html>
'''.format(rows_analyzed=7)  # Update with actual row count

report_path = STAGE5_OUT_DIR / 'report.html'
report_path.write_text(html_content)
print(f"✅ Saved HTML report: {report_path}")
```
```

PHASE 3: REPORT RESULTS (1 tool call)
--------------------------------------
Call save_visualization_report() with JSON listing ALL separate PNG files:
```json
{
  "plan_id": "PLAN-TSK-002",
  "task_category": "descriptive",
  "visualizations": [
    "stage5_out/quantity_trend.png",
    "stage5_out/quantity_by_variety.png",
    "stage5_out/value_trend_inr.png",
    "stage5_out/quantity_vs_value.png"
  ],
  "html_report": "stage5_out/report.html",
  "summary": "Created 4 separate, high-quality visualizations showing rice export trends from 2018-2025. Each chart focuses on one clear insight with proper labels and formatting.",
  "insights": [
    "Export quantities show significant year-over-year variation with peak in 2024-25",
    "One rice variety dominates total exports, accounting for over 60% of quantity",
    "Value trends indicate price variations beyond simple quantity changes",
    "Clear seasonality patterns visible in the data"
  ]
}
```

═══════════════════════════════════════════════════════════════
VISUALIZATION BEST PRACTICES
═══════════════════════════════════════════════════════════════

**Colors**: Use colorblind-friendly palettes
```python
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
sns.set_palette(sns.color_palette("colorblind"))
```

**Labels**: Always label axes, add titles, include units
```python
plt.xlabel('Year', fontsize=12)
plt.ylabel('Export Quantity (Tonnes)', fontsize=12)
plt.title('Export Trends 2018-2024', fontsize=14, fontweight='bold')
```

**Legends**: Make them clear and informative
```python
plt.legend(title='Rice Variety', loc='best', frameon=True)
```

**Resolution**: Save high-quality images
```python
plt.savefig(path, dpi=300, bbox_inches='tight')
```

**Layout**: Use tight_layout() or constrained_layout
```python
plt.tight_layout()
# or
fig = plt.figure(constrained_layout=True)
```

═══════════════════════════════════════════════════════════════
CHART SELECTION GUIDE
═══════════════════════════════════════════════════════════════

**Trends over time**: Line plot, area chart
**Comparisons**: Bar chart, grouped bar chart
**Distributions**: Histogram, box plot, violin plot
**Correlations**: Scatter plot, heatmap
**Proportions**: Pie chart, stacked bar chart
**Relationships**: Scatter plot with regression line
**Hierarchies**: Treemap, sunburst
**Geospatial**: Choropleth map (if location data)

═══════════════════════════════════════════════════════════════
EXAMPLE: DESCRIPTIVE TASK VISUALIZATION
═══════════════════════════════════════════════════════════════

Task: Export trend analysis

Your workflow:
1. load_stage4_result() - get summary statistics
2. create_visualizations() - load data, create 4-panel overview
3. create_visualizations() - create trend analysis plots
4. create_visualizations() - generate HTML report
5. save_visualization_report() - document all outputs

═══════════════════════════════════════════════════════════════
EXAMPLE: PREDICTIVE TASK VISUALIZATION
═══════════════════════════════════════════════════════════════

Task: Yield prediction

Your workflow:
1. load_stage4_result() - get predictions and metrics
2. create_visualizations() - actual vs predicted plots
3. create_visualizations() - residual analysis
4. create_visualizations() - error distribution
5. create_visualizations() - generate comprehensive HTML report
6. save_visualization_report() - document insights

═══════════════════════════════════════════════════════════════

Remember:
- Create clear, publication-quality visualizations
- Tell a story with your charts
- Provide actionable insights
- Save high-resolution outputs
- Generate a comprehensive HTML report
- End by calling save_visualization_report()

Your success = Beautiful visuals + Meaningful insights + Report saved."""

# ===========================
# LangGraph
# ===========================

def agent_node(state: MessagesState) -> Dict[str, List[BaseMessage]]:
    """LLM agent step."""
    response: AIMessage = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}


tool_node = ToolNode(TOOLS)


def should_continue(state: MessagesState) -> str:
    """Route based on tool calls."""
    last = state["messages"][-1]
    if isinstance(last, AIMessage) and last.tool_calls:
        return "tools"
    return END


builder = StateGraph(MessagesState)
builder.add_node("agent", agent_node)
builder.add_node("tools", tool_node)
builder.set_entry_point("agent")
builder.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
builder.add_edge("tools", "agent")

memory = MemorySaver()
stage5_app = builder.compile(checkpointer=memory)


# ===========================
# Runner
# ===========================

def run_stage5_for_result(result_id: str, max_rounds: int = 20, debug: bool = True):
    """Create visualizations for a Stage 4 result."""
    
    system_msg = SystemMessage(content=STAGE5_SYSTEM_PROMPT)
    human_msg = HumanMessage(
        content=(
            f"Create comprehensive visualizations for: '{result_id}'\n\n"
            f"Workflow:\n"
            f"1. load_stage4_result('{result_id}')\n"
            f"2. Understand the task and output files\n"
            f"3. FIRST: create_visualizations() to INSPECT data (print columns, head, dtypes)\n"
            f"4. SECOND: create_visualizations() to make plots using ACTUAL column names\n"
            f"5. Generate HTML report\n"
            f"6. save_visualization_report() with all outputs\n\n"
            f"CRITICAL: Never assume column names! Inspect first, then plot.\n"
            f"Your success = Data inspected + Beautiful charts + Insights + Report saved."
        )
    )

    state: MessagesState = {"messages": [system_msg, human_msg]}

    if not debug:
        return stage5_app.invoke(state, config={"configurable": {"thread_id": f"stage5-{result_id}"}})

    print("=" * 80)
    print(f"🚀 STAGE 5: Creating visualizations for {result_id}")
    print("=" * 80)

    final_state = None
    prev_len = 0
    round_num = 0

    for curr_state in stage5_app.stream(
        state,
        config={"configurable": {"thread_id": f"stage5-{result_id}"}},
        stream_mode="values",
    ):
        msgs = curr_state["messages"]
        new_msgs = msgs[prev_len:]

        for m in new_msgs:
            if isinstance(m, SystemMessage):
                print("\n" + "─" * 80)
                print("💻 [SYSTEM]")
                print("─" * 80)
                print(m.content[:500] + "..." if len(m.content) > 500 else m.content)

            elif isinstance(m, HumanMessage):
                print("\n" + "─" * 80)
                print("👤 [USER]")
                print("─" * 80)
                print(m.content)

            elif isinstance(m, AIMessage):
                round_num += 1
                print("\n" + "═" * 80)
                print(f"🤖 [AGENT - Round {round_num}]")
                print("═" * 80)

                if m.content:
                    print("\n💭 Reasoning:")
                    # 👉 print full reasoning, no truncation
                    print(m.content)

                if m.tool_calls:
                    print("\n🔧 Tool Calls:")
                    for tc in m.tool_calls:
                        name = getattr(tc, "name", tc.get("name", "UNKNOWN"))
                        args = getattr(tc, "args", tc.get("args", {}))

                        print(f"\n  📌 {name}")
                        for k, v in args.items():
                            # 👉 print full args, especially the generated code
                            print(f"     {k}: {v}")


            elif isinstance(m, ToolMessage):
                print("\n" + "─" * 80)
                print(f"📊 [TOOL: {m.name}]")
                print("─" * 80)
                print(str(m.content))

        prev_len = len(msgs)
        final_state = curr_state
        
        if round_num >= max_rounds:
            print(f"\n⚠️  Reached max rounds ({max_rounds})")
            break

    print("\n" + "=" * 80)
    print(f"✅ Complete - {round_num} rounds")
    print("=" * 80)

        # 💾 Save full conversation trace to JSON so you can inspect thoughts later
    if final_state is not None:
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        trace_path = STAGE5_OUT_DIR / f"stage5_trace_{result_id}_{ts}.json"

        try:
            trace = messages_to_dict(final_state["messages"])
            trace_path.write_text(json.dumps(trace, indent=2))
            print(f"\n📝 Saved full Stage 5 agent trace to: {trace_path}")
        except Exception as e:
            print(f"\n⚠️ Failed to save trace: {e}")
            
    return final_state


# ===========================
# Main
# ===========================

if __name__ == "__main__":
    import sys
    
    if len(sys.argv) < 2:
        print("Usage: python stage5_visualizer.py <result_id>")
        print("Example: python stage5_visualizer.py execution_PLAN-TSK-002_20250101_120000.json")
        print("\nAvailable results:")
        for result in _list_stage4_results_impl():
            print(f"  - {result}")
        sys.exit(1)
    
    result_id = "execution_PLAN-TSK-002_20251118_082013.json"
    print(f"\n🎯 Creating visualizations for: {result_id}\n")
    
    result = run_stage5_for_result(result_id, debug=True)
    
    # Check for visualization reports
    reports = sorted(STAGE5_OUT_DIR.glob("visualization_report_*.json"))
    if reports:
        latest = reports[-1]
        print(f"\n✅ Visualization report: {latest}")
        report_data = json.loads(latest.read_text())
        print("\nCreated visualizations:")
        for viz in report_data.get('visualizations', []):
            print(f"  - {viz}")
        if report_data.get('html_report'):
            print(f"\nHTML Report: {report_data['html_report']}")
    else:
        print("\n⚠️  No visualization report saved")